Loading the model in 

In [3]:
import torch
import pandas as pd
import joblib

from torch import nn 

from common_utils import MLPBuilder
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import MinMaxScaler


In [4]:
# we trained the model with 8 features
model = MLPBuilder(no_features=8, layers=[256, 512 , 256, 128])

# Change this when loading a new model
# model.load_state_dict(torch.load('models/NN_model.pth'))
# model.load_state_dict(torch.load('models/NN_model_augmented.pth'))
model_path = './models/Grid_based'
model.load_state_dict(torch.load(model_path+'/NN_model_new.pth'))

C:\Users\randa\AppData\Local\Temp\ipykernel_3596\3951161975.py:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path+'/NN_model_new.pt

<All keys matched successfully>

## Loading test data in 

In [5]:
file_path = './Data/23_Dec_Readings/combined_data.csv'
cleaned_df = pd.read_csv(file_path)

In [6]:
cleaned_df

,Tx_0 RSSI,Tx_0 Batt,Tx_0 Time,Tx_1 RSSI,Tx_1 Batt,Tx_1 Time,Tx_2 RSSI,Tx_2 Batt,Tx_2 Time,Tx_3 RSSI,...,Tx_5 Batt,Tx_5 Time,Tx_6 RSSI,Tx_6 Batt,Tx_6 Time,Tx_7 RSSI,Tx_7 Batt,Tx_7 Time,X_Coord,Y_Coord
0,127,NaN,0.0,-77,4.11,0.0,-73,4.09,0.0,-75,...,4.05,0.0,-50,4.10,0.0,-75,NaN,0.0,7,0
1,127,NaN,24.0,-74,4.11,24.0,-73,NaN,28.0,-76,...,4.06,28.0,-49,4.10,29.0,-76,4.04,25.0,7,0
2,-82,NaN,53.0,-79,NaN,58.0,-73,4.09,58.0,-79,...,4.06,58.0,-49,4.11,60.0,-78,4.06,57.0,7,0
3,-82,4.11,81.0,-73,NaN,83.0,-73,4.09,84.0,-75,...,NaN,88.0,-49,4.09,90.0,-77,4.06,86.0,7,0
4,-81,NaN,111.0,-72,4.11,111.0,-72,NaN,115.0,-78,...,4.06,111.0,-48,4.11,111.0,-76,4.04,108.0,7,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
895,-75,NaN,390.0,-66,4.10,389.0,-81,4.08,383.0,-63,...,4.05,381.0,-74,4.11,382.0,-64,4.05,383.0,3,1
896,-76,NaN,414.0,-65,4.11,414.0,-81,4.09,408.0,-63,...,NaN,407.0,-75,4.10,408.0,-63,4.06,409.0,3,1
897,-77,4.11,436.0,-65,4.11,434.0,-79,4.09,429.0,-64,...,4.06,422.0,-76,4.10,421.0,-63,NaN,426.0,3,1
898,-77,4.11,453.0,-65,4.11,452.0,-81,NaN,451.0,-63,...,4.05,443.0,-76,4.11,443.0,-65,NaN,447.0,3,1


In [7]:
batt_columns = [col for col in cleaned_df.columns if 'Batt' in col]
time_columns = [col for col in cleaned_df.columns if 'Time' in col]
columns_to_drop = batt_columns+time_columns
RSSI_columns = cleaned_df.drop(columns=columns_to_drop)


#Train the model without NAN values
# cleaned_df = RSSI_columns.dropna()

#Convert NaN values to 0
# cleaned_df = RSSI_columns.fillna(127)

In [8]:
cleaned_df

,Tx_0 RSSI,Tx_0 Batt,Tx_0 Time,Tx_1 RSSI,Tx_1 Batt,Tx_1 Time,Tx_2 RSSI,Tx_2 Batt,Tx_2 Time,Tx_3 RSSI,...,Tx_5 Batt,Tx_5 Time,Tx_6 RSSI,Tx_6 Batt,Tx_6 Time,Tx_7 RSSI,Tx_7 Batt,Tx_7 Time,X_Coord,Y_Coord
0,127,NaN,0.0,-77,4.11,0.0,-73,4.09,0.0,-75,...,4.05,0.0,-50,4.10,0.0,-75,NaN,0.0,7,0
1,127,NaN,24.0,-74,4.11,24.0,-73,NaN,28.0,-76,...,4.06,28.0,-49,4.10,29.0,-76,4.04,25.0,7,0
2,-82,NaN,53.0,-79,NaN,58.0,-73,4.09,58.0,-79,...,4.06,58.0,-49,4.11,60.0,-78,4.06,57.0,7,0
3,-82,4.11,81.0,-73,NaN,83.0,-73,4.09,84.0,-75,...,NaN,88.0,-49,4.09,90.0,-77,4.06,86.0,7,0
4,-81,NaN,111.0,-72,4.11,111.0,-72,NaN,115.0,-78,...,4.06,111.0,-48,4.11,111.0,-76,4.04,108.0,7,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
895,-75,NaN,390.0,-66,4.10,389.0,-81,4.08,383.0,-63,...,4.05,381.0,-74,4.11,382.0,-64,4.05,383.0,3,1
896,-76,NaN,414.0,-65,4.11,414.0,-81,4.09,408.0,-63,...,NaN,407.0,-75,4.10,408.0,-63,4.06,409.0,3,1
897,-77,4.11,436.0,-65,4.11,434.0,-79,4.09,429.0,-64,...,4.06,422.0,-76,4.10,421.0,-63,NaN,426.0,3,1
898,-77,4.11,453.0,-65,4.11,452.0,-81,NaN,451.0,-63,...,4.05,443.0,-76,4.11,443.0,-65,NaN,447.0,3,1


In [9]:
X = cleaned_df[['Tx_0 RSSI', 'Tx_1 RSSI', 'Tx_2 RSSI', 'Tx_3 RSSI', 'Tx_4 RSSI', 'Tx_5 RSSI', 'Tx_6 RSSI', 'Tx_7 RSSI']]
Y = cleaned_df[['X_Coord', 'Y_Coord']]

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

scaler_X = joblib.load(model_path+'/scaler_X.pkl')
# scaler_Y = joblib.load('./models/scales/scaler_Y.pkl')

X_train_scaled = scaler_X.transform(X_train)
X_test_scaled = scaler_X.transform(X_test)
# Y_test_scaled = scaler_Y.transform(Y)

X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)
# Y_test_tensor = torch.tensor(Y_test_scaled, dtype=torch.float32)

Y_train_array = Y_train.to_numpy()  # Ensure these are NumPy arrays
Y_test_array = Y_test.to_numpy()

Y_train_tensor_x = torch.tensor(Y_train_array[:, 0], dtype=torch.long)  # X_Coord
Y_train_tensor_y = torch.tensor(Y_train_array[:, 1], dtype=torch.long)  # Y_Coord

Y_test_tensor_x = torch.tensor(Y_test_array[:, 0], dtype=torch.long)  # X_Coord
Y_test_tensor_y = torch.tensor(Y_test_array[:, 1], dtype=torch.long)  # Y_Coord

# Create DataLoaders
train_dataset = TensorDataset(X_train_tensor, Y_train_tensor_x, Y_train_tensor_y)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Create DataLoader for the test set
test_dataset = TensorDataset(X_test_tensor, Y_test_tensor_x, Y_test_tensor_y)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


# Transfer Learning

In [10]:
from common_utils import train_loop, test_loop
import torch.optim as optim

for param in model.mlp_stack[-3].parameters(): 
    param.requires_grad = True

# Unfreeze the output layers
for param in model.output_x.parameters():
    param.requires_grad = True
for param in model.output_y.parameters():
    param.requires_grad = True

# Now the optimizer will only optimize the parameters that have requires_grad=True
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.001)

# Train and evaluate the model
epochs = 100
for epoch in range(epochs):
    train_loss, train_accuracy = train_loop(train_loader, model, nn.CrossEntropyLoss(), nn.CrossEntropyLoss(), optimizer)
    test_loss, test_accuracy = test_loop(test_loader, model, nn.CrossEntropyLoss(), nn.CrossEntropyLoss())
    print(f"Epoch {epoch + 1}/{epochs} - Train Loss: {train_loss:.4f}, Test Accuracy: {test_accuracy:.2f}%")


Epoch 1/100 - Train Loss: 6.1215, Test Accuracy: 11.11%
Epoch 2/100 - Train Loss: 3.4891, Test Accuracy: 13.33%
Epoch 3/100 - Train Loss: 3.0453, Test Accuracy: 24.44%
Epoch 4/100 - Train Loss: 2.8344, Test Accuracy: 31.11%
Epoch 5/100 - Train Loss: 2.6014, Test Accuracy: 34.44%
Epoch 6/100 - Train Loss: 2.4876, Test Accuracy: 47.22%
Epoch 7/100 - Train Loss: 2.2149, Test Accuracy: 51.11%
Epoch 8/100 - Train Loss: 2.0484, Test Accuracy: 51.11%
Epoch 9/100 - Train Loss: 1.9745, Test Accuracy: 50.56%
Epoch 10/100 - Train Loss: 1.8641, Test Accuracy: 55.00%
Epoch 11/100 - Train Loss: 1.7134, Test Accuracy: 55.56%
Epoch 12/100 - Train Loss: 1.6977, Test Accuracy: 60.56%
Epoch 13/100 - Train Loss: 1.6315, Test Accuracy: 62.22%
Epoch 14/100 - Train Loss: 1.5360, Test Accuracy: 63.33%
Epoch 15/100 - Train Loss: 1.3886, Test Accuracy: 66.11%
Epoch 16/100 - Train Loss: 1.4246, Test Accuracy: 66.67%
Epoch 17/100 - Train Loss: 1.2681, Test Accuracy: 66.67%
Epoch 18/100 - Train Loss: 1.2350, Test 

In [11]:
loss_fn_x = nn.CrossEntropyLoss()
loss_fn_y = nn.CrossEntropyLoss()

test_loss, test_accuracy = test_loop(test_loader, model, loss_fn_x, loss_fn_y)

print(f"Test Loss: {test_loss:.4f}")
print(f"Test Combined Accuracy: {test_accuracy:.2f}%")


Test Loss: 0.5873
Test Combined Accuracy: 90.56%


In [12]:
model.eval()

# Choose a sample input from the test set
inputs, targets_x, targets_y = next(iter(test_loader))  # Get the first batch from the test_loader

# Select only the first 10 samples from the batch
inputs = inputs[:10]
actual_targets = torch.stack((targets_x[:10], targets_y[:10]), dim=1)  # Combine actual x and y coordinates

# Perform the forward pass
with torch.no_grad():
    logits_x, logits_y = model(inputs)  # Outputs from the model

    # For classification, use argmax to get the predicted class
    preds_x = torch.argmax(logits_x, dim=1)  # Predicted x-coordinates
    preds_y = torch.argmax(logits_y, dim=1)  # Predicted y-coordinates

# Combine predictions for display
predictions = torch.stack((preds_x, preds_y), dim=1)  # Shape: (10, 2)

# Print results
print("Predicted X and Y coordinates (grid indices):")
print(predictions.numpy())  # Convert to numpy for better readability

print("\nActual X and Y coordinates (grid indices):")
print(actual_targets.numpy())


Predicted X and Y coordinates (grid indices):
[[7 6]
 [2 4]
 [7 7]
 [0 6]
 [7 2]
 [0 2]
 [6 1]
 [6 4]
 [7 6]
 [6 6]]

Actual X and Y coordinates (grid indices):
[[7 6]
 [2 4]
 [7 7]
 [0 6]
 [7 2]
 [0 2]
 [6 1]
 [6 4]
 [7 6]
 [6 6]]
